# Flask + WhyLabs

In [2]:
import os
import json
import pandas as pd
import requests
from joblib import dump
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import boto3

## Download Iris dataset

In [4]:
!chmod +x download_iris.sh && ./download_iris.sh

  0%|                                               | 0.00/3.60k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 3.60k/3.60k [00:00<00:00, 1.91MB/s]
Archive:  iris.zip
  inflating: Iris.csv                
  inflating: database.sqlite         


## Train data

In [5]:
data = pd.read_csv('dataset/iris.csv')
# Separating the independent variables from dependent variables
X = data.drop(['Id', 'Species'], axis=1)
y = data['Species']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [7]:
# Train a classifier
print("Train started...")
model = SVC()
model.fit(x_train, y_train)
print("Train ended...")
# Save the model
dump(model, 'model.joblib')
print("Model saved!")

Train started...
Train ended...
Model saved!


## Build the docker image

In [17]:
os.system("docker build --build-arg PYTHON_VERSION=3.7 -t whylabs-flask .")

#1 [internal] load build definition from Dockerfile
#1 sha256:cc9f986c784f5229328f144b949daf2351dd81ad6af57fa87360d2f2702006a3
#1 transferring dockerfile: 37B 0.0s done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:ecfa6189d3f8cf892daefc7e9ce2f514b383cddc0210ac5856856aad0e25c586
#2 transferring context: 34B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/python:3.7
#3 sha256:2e83735093f2e645c7660e349795ae80186113afc054af8bdc7501e99da12131
#3 DONE 0.7s

#4 [1/5] FROM docker.io/library/python:3.7@sha256:2f8af5fa73dd59a8d3b65fe56273bdd5ab9a991bc92ebf434960183e22d6b91a
#4 sha256:dfa2226fef6538e7b83826fbcb1d1db44561108fab944ef5abeb5eb619b92757
#4 DONE 0.0s

#6 [internal] load build context
#6 sha256:23520de576c2c2b821f8d862e41ab546eb44ef1265021f5002b54f39458882df
#6 transferring context: 39.10kB 0.0s done
#6 DONE 0.0s

#5 [2/5] WORKDIR /usr/src/app
#5 sha256:d471b7f9ee179acd5a21036436f5c872e88c081b23b5c5ed2e2fd87d0de36d23
#5 CACHED

#7 [3/5] COPY requirements

0

## Run the container to serve the API

Open a terminal and execute the following command:

```bash
docker run --rm -p 5000:5000 -v $(pwd):/app  whylabs-flask
```

## Test endpoint

In [20]:
# Payload for /invocations endpoint
payload = json.dumps({
    "sepal_length_cm": 5.1,
    "sepal_width_cm": 3.5,
    "petal_length_cm": 1.4,
    "petal_width_cm": 0.2
})

In [22]:
requests.post(
    "http://0.0.0.0:5000/predict", 
    data=payload
)

KeyboardInterrupt: 